# Demos: Lecture 2

## Demo 1: QNodes

<img src="fig/xzh.png" width=300>

In [4]:
import pennylane as qml
import numpy as np

In [5]:
dev = qml.device("default.qubit", wires=1, shots=50)

In [7]:
def cool_function():
    qml.Hadamard(wires=0)
    qml.PauliZ(wires=0)
    qml.PauliX(wires=0)
    return qml.sample()

In [8]:
my_qnode = qml.QNode(cool_function, dev)

In [9]:
my_qnode()

tensor([0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0,
        0, 1, 1, 1, 0, 0], requires_grad=True)

In [10]:
results = my_qnode()

In [11]:
np.mean(results)

tensor(0.52, requires_grad=True)

## Exercise 1: relative phases

Implement the circuit in the picture. Run it on a device with 1000 shots, and try two different values of $\theta$. How does $\theta$ affect the measurement outcome probabilites?

<img src="fig/exercise-circuit.png" width=600>

In [12]:
dev = qml.device("default.qubit", wires=1, shots=1000)

In [13]:
def circuit(theta):
    qml.Hadamard(wires=0)
    qml.RZ(theta, wires=0)
    return qml.sample()

In [14]:
circuit_qnode = qml.QNode(circuit, dev)

In [15]:
results_03 = circuit_qnode(0.3)

In [16]:
results_other = circuit_qnode(0.4)

In [17]:
np.mean(results_03)

tensor(0.477, requires_grad=True)

In [18]:
np.mean(results_other)

tensor(0.518, requires_grad=True)

## Exercise 2: state preparation

Write a QNode that prepares the quantum state

$$
\begin{equation*}
|\psi \rangle = \frac{\sqrt{3}}{2}|0 \rangle  - \frac{1}{2} e^{i\frac{5}{4}}| 1\rangle 
\end{equation*}
$$

Run it on an ideal simulated device (do not set `shots`), and return `qml.state()`. 

In [19]:
dev = qml.device("default.qubit", wires=1)

In [44]:
def prepare_state():
    # The RY gets us the correct amplitudes of the coefficients
    qml.RY(-np.pi/3, wires=0)
    
    # The RZ will add the phases; notice that we are adding π to the phase.
    # This is becaues of the (-) out front, as -e^{i5/4} = e^{i(π + 5/4)}.
    qml.RZ(np.pi + 5/4, wires=0)
    return qml.state()

In [45]:
qnode = qml.QNode(prepare_state, dev)

In [46]:
qnode()

tensor([-0.5067091 -0.70231466j,  0.29254864-0.40548156j], requires_grad=True)

Note that there is a complex part in the first element ($\alpha$). This is because of the form of $RZ(\theta)$:

\begin{equation}
RZ(\theta) = \begin{pmatrix} e^{-i\theta/2} & 0 \\ 0 & e^{i\theta / 2} \end{pmatrix}
\end{equation}

So the $\vert 0 \rangle$ state has an extra phase of $e^{-i\theta/2}$. But that's okay: we can pull out the complex part as a *global phase* and see what we have left.

In [47]:
state_with_global_phase = qnode()
global_phase = np.angle(state_with_global_phase[0]) # Get the angle of complex part
without_global_phase = state_with_global_phase / np.exp(1j * global_phase)

In [48]:
state_without_global_phase

tensor([0.8660254 -0.j        , 0.15766118+0.47449231j], requires_grad=True)

The first element matches now; let's check the second by looking at its magnitude and phase.

In [49]:
np.abs(state_without_global_phase[1])

tensor(0.5, requires_grad=True)

In [50]:
np.angle(state_without_global_phase[1])

tensor(1.25, requires_grad=True)

## Demo 2: universality of Pauli rotations

The Hadamard gate three ways.

In [ ]:
def hadamard():
    qml.Hadamard(wires=0)

def hadamard_with_rzrx():
    qml.RZ(np.pi/2, wires=0)
    qml.RX(np.pi/2, wires=0)
    qml.RZ(np.pi/2, wires=0)
    
def hadamard_with_rot():
    qml.Rot(np.pi, np.pi/2, 0, wires=0)